In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))
import plotly.graph_objects as go

# Bitcoin Trading Robot

# 1426% Wow

### How to create trading robot in 5 minutes on Kaggle to BackTest your strategy

#### There are a lot of people among you who have made very cool progress in neural networks, 
#### I am still in the process of studying, but I decided to share with you my knowledge where I am an expert (winner of two hackathons of trading robots) 
#### and specially for you I created a trading robot which can be run on Kaggle on your data, 
#### in which you can add your knowledge of neural networks and create your own trading robots using neural networks for backtesting on history.
-- Just put your logic in class method ```def next(self):```

# Trading Strategy

#### For demonstration purposes, let's buy BTC when the fast SMA crosses the slower SMA and sell in reverse
##### P.S. We will use timeframe D1 and prices of Bitcoin

In [ ]:
# you can change config here)))
ticker = "BTCUSDT"
timeframe = "D1"
fast_sma_period = 20
slow_sma_period = 60

# Load Data

In [ ]:
df = pd.read_csv(f"/kaggle/input/crypto-coins-prices-ohlcv/{timeframe}/{ticker}_{timeframe}.csv", parse_dates=['datetime'])
# df.set_index('datetime', inplace=True)

In [ ]:
df

# Chart

In [ ]:
figure = go.Figure(

    data = [
            go.Candlestick(
            x = df["datetime"],
            low = df['low'],
            high = df['high'],
            close = df['close'],
            open = df['open'],
            increasing_line_color = 'green',
            decreasing_line_color = 'black'
        )
    ]
)
figure.update_layout(
        title = 'Bitcoin Stock Prices',
        yaxis_title = 'Bitcoin stock prices ($)',
        xaxis_title = 'Date')
figure.show()

In [ ]:
# this is need to download stock data to Backtrader system for Backtest on history
df.set_index('datetime', inplace=True)

# Libraries

#### For backtest our strategy we will use lib backtrader (it's updated version is available on my github, the author of this library has approved all my changes)

In [ ]:
!pip install backtrader

# Code of Trading Robot

In [ ]:
import backtrader as bt
import backtrader.analyzers as btanalyzers

In [ ]:
# code of trading robot - your trading conditions - when to buy and when to sell
class SMACrossStrategy(bt.Strategy):
    """ Live strategy demonstration with SMA """
    params = (
        ("fast_sma_period", 50),
        ("slow_sma_period", 200),
        ('timeframe', ''),
    )    

    def __init__(self):
        self.fast_sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.fast_sma_period)
        self.slow_sma = bt.indicators.SimpleMovingAverage(self.data.close, period=self.params.slow_sma_period)

    def next(self):
        """The arrival of a new ticker bar"""
        for data in self.datas:  # We run through all the requested bars of all tickers
            ticker = data._name
            _interval = self.p.timeframe
            _date = bt.num2date(data.datetime[0])
            print('{} / {} [{}] - Open: {}, High: {}, Low: {}, Close: {}, Volume: {}'.format(bt.num2date(data.datetime[0]), data._name, _interval, data.open[0], data.high[0], data.low[0], data.close[0], data.volume[0], ))
            
            # Check for crossover
            if self.fast_sma > self.slow_sma and self.position.size == 0:
                free_money = self.broker.getcash()
                price = data.close[0]  # by close price
                size = (free_money / price) * 0.9  # 90% of free money # 0.1 => 10% of free money
                self.buy(data=data, exectype=bt.Order.Limit, price=price, size=size)

            # Check for crossunder
            elif self.fast_sma < self.slow_sma and self.position.size > 0:
                self.close()

    def notify_order(self, order):
        """Changing the status of the order"""
        order_data_name = order.data._name  # The name of the ticker from the order
        print("*" * 50)
        self.log(f'Order number {order.ref} {order.info["order_number"]} {order.getstatusname()} {"Buy" if order.isbuy() else "Sell"} {order_data_name} {order.size} @ {order.price}')
        if order.status == bt.Order.Completed:  # If the order is fully executed
            if order.isbuy():  # if the order is buy
                self.log(f'Buy {order_data_name} Price: {order.executed.price:.2f}, Val: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}')
            else:  # if the order is sell
                self.log(f'Sell {order_data_name} Price: {order.executed.price:.2f}, Val: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}')
        print("*" * 50)

    def notify_trade(self, trade):
        """Changing the position status"""
        if trade.isclosed:  # If the position is closed
            self.log(f'Profit on a closed position {trade.getdataname()} Total={trade.pnl:.2f}, Comm={trade.pnlcomm:.2f}')

    def log(self, txt, dt=None):
        """Output a date string to the console"""
        dt = bt.num2date(self.datas[0].datetime[0]) if not dt else dt  # The set date or the date of the current bar
        print(f'{dt.strftime("%d.%m.%Y %H:%M")}, {txt}')  # Output the date and time with the specified text to the console

In [ ]:
# Create a cerebro engine
cerebro = bt.Cerebro()

# Set the initial cash amount for backtesting
cerebro.broker.set_cash(100000)

# Set commission - You can adjust this based on your preferences or use other commission schemes
cerebro.broker.setcommission(commission=0.001)

In [ ]:
# Add a data feed
data = bt.feeds.PandasData(dataname=df, name=ticker)
cerebro.adddata(data)

In [ ]:
# Add the strategy
cerebro.addstrategy(SMACrossStrategy, timeframe=timeframe, fast_sma_period=fast_sma_period, slow_sma_period=slow_sma_period)

In [ ]:
# Add Finance metrics of quality our strategy on historical data (backtest and results)
cerebro.addanalyzer(btanalyzers.SQN, _name='SQN')
cerebro.addanalyzer(btanalyzers.VWR, _name='VWR', fund=True)
cerebro.addanalyzer(btanalyzers.TimeDrawDown, _name='TDD', fund=True, timeframe=bt.TimeFrame.Days)
cerebro.addanalyzer(btanalyzers.DrawDown, _name='DD', fund=True)
cerebro.addanalyzer(btanalyzers.Returns, _name='R', fund=True, timeframe=bt.TimeFrame.Days)
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='AR', )
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='SR')
cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='TradeAnalyzer')

#### Run backtest strategy

In [ ]:
# Print the starting cash amount
print(f"Starting Portfolio Value: {cerebro.broker.getvalue():.2f}")

In [ ]:
# Run the strategy
results = cerebro.run()  # run
thestrat = results[0]  # get results

In [ ]:
# Print the final cash amount
print('Ending Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('Remaining available funds: %.2f' % cerebro.broker.getcash())
print('Assets in the amount of: %.2f' % (cerebro.broker.getvalue() - cerebro.broker.getcash()))
print()
print('SQN: ', thestrat.analyzers.SQN.get_analysis())
print('VWR: ', thestrat.analyzers.VWR.get_analysis())
print('TDD: ', thestrat.analyzers.TDD.get_analysis())
print('DD: ', thestrat.analyzers.DD.get_analysis())
print('AR: ', thestrat.analyzers.AR.get_analysis())
print('Profitability: ', thestrat.analyzers.R.get_analysis())

# Proifit

#### Starting Portfolio Value: 100 000.00
#### Ending Portfolio Value: 1 526 360.38 => **+1426%**

# The End

#### Feel free to leave comment. Thanks.